In [1]:
import dask
from dask.distributed import Client

In [2]:
client = Client()

In [3]:
client

Client Scheduler: tcp://127.0.0.1:38175 Dashboard: http://127.0.0.1:8787,Cluster Workers: 12 Cores: 12 Memory: 30.32 GB


In [4]:
import dask.dataframe as dd

In [5]:
HiC_chr1=dd.read_csv('/home/alberto/DATA/GMUDOsOdieciK/chr1.txt',sep='\t',header=None,names=['win1', 'win2', 'contactP'])

In [8]:
sortHc=HiC_chr1.set_index('win1')

In [10]:
sortHc.head()

,win2,contactP
win1,,
0,87650000,NaN
10000,38270000,39.849964
10000,36330000,44.781060
10000,35830000,69.356705
10000,35480000,55.603490


In [12]:
sortHc=sortHc.reset_index()

In [15]:
sortHc['distance']=sortHc.apply(lambda x: int(abs(x['win1']-x['win2'])),axis=1 )

In [16]:
tempHc=sortHc.rename(columns={'win1':'win2','win2':'win1'})

In [17]:
sortHc[['contactP','distance']].corr()

,contactP,distance
npartitions=1,,
contactP,float64,float64
distance,...,...


In [18]:
Big=dd.concat([sortHc,tempHc])

In [19]:
def openGC(path,resolution):
    import gzip
    def window(ising,lung):
        L=len(ising)

        GCdens=[]
        distances = range((L/lung)+1)
        for i in distances:
            temp=ising[i*lung:(i+1)*lung]

            NP=temp.count('+')
            NM=temp.count('-')
            if NP+NM != 0: 
                NPmed=NP/float(NP+NM)
            else: NPmed = float('nan')
            GCdens.append(NPmed)
        #print len(GCdens)
        return GCdens

    path
    with gzip.open(path, 'rb') as f:
        ising = f.read()

    isingN = '\n'.join(ising.split('\n')[1:])
    ising=isingN.replace('\n','').replace('\r','')
    ising=ising.replace('A','-').replace('T','-').replace('G','+').replace('C','+').replace('a','-').replace('t','-').replace('g','+').replace('c','+')
    #L=len(ising)

    muVec=window(ising,resolution)   #change resolution here
    return muVec

In [20]:
ch1_fa=openGC('/home/alberto/Downloads/HI-C/Reference Genomes/hg19/chr1.fa.gz',10**4)

In [23]:
import pandas as pd

In [25]:
GC=pd.DataFrame(ch1_fa,columns=['GC_content']).reset_index()
GC['index']=GC['index']*5*10**5
GC.rename(columns ={'index': 'window'},inplace=True)

In [26]:
Big.drop_duplicates(inplace=True)

,contactP,distance,win1,win2
npartitions=1,,,,
None,float64,int64,int64,int64
None,...,...,...,...


In [27]:
sortHc=Big

In [31]:
tmpMerge=dd.merge(sortHc,GC,left_on='win1',right_on='window',left_index=True)
tmpMerge=tmpMerge.rename(columns ={'GC_content': 'GC_content1'})
df=dd.merge(tmpMerge,GC,left_on='win2',right_on='window',left_index=True)
df=df.rename(columns ={'GC_content': 'GC_content2'})
cleanDf=df.drop(['window_x','window_y'], axis=1)
cleanDf['distance']=cleanDf['distance'].astype(int)

In [32]:
cleanDf['GC_diff']=cleanDf.apply(lambda x: abs(x['GC_content1']-x['GC_content2']),axis=1 )

In [ ]:
cleanDf[['distance','contactP','GC_diff']].corr().compute()